In [3]:
import os

# --- Setup for Task 1: Creating dummy scripts to execute ---
script_content_1 = "print('--> Script 1 is executing: Hello World')"
script_content_2 = "import time; print('--> Script 2 is executing: Calculating...'); time.sleep(1); print('--> Script 2 finished.')"

with open("batch_script_1.py", "w") as f:
    f.write(script_content_1)

with open("batch_script_2.py", "w") as f:
    f.write(script_content_2)

# --- Setup for Task 2: Creating a worker file for Multiprocessing ---
# We create this external file to avoid 'AttributeError' in Jupyter Notebooks
worker_content = """
import time
import logging

def process_task(name):
    logger = logging.getLogger()
    # Configure logger inside process if not inheriting (safety check)
    if not logger.handlers:
        logging.basicConfig(filename='system_log.txt', level=logging.INFO,
                            format='%(asctime)s - %(processName)s - %(message)s')
    
    logger.info(f"{name} started")
    # Simulate work
    time.sleep(1) 
    logger.info(f"{name} terminated")
"""

with open("task_utils.py", "w") as f:
    f.write(worker_content)

print("Setup complete: Helper files created successfully.")

Setup complete: Helper files created successfully.


In [4]:
# Task 1 - Batch Processing Simulation

import subprocess
import sys

def run_batch_processing():
    print("--- Task 1: Batch Processing Simulation ---")
    
    # List of scripts created in Cell 1
    scripts_to_run = ['batch_script_1.py', 'batch_script_2.py']
    
    for script in scripts_to_run:
        print(f"Main System: Launching {script}...")
        try:
            # Using sys.executable ensures we use the same python interpreter as the notebook
            subprocess.run([sys.executable, script], check=True)
            print(f"Main System: {script} executed successfully.\n")
        except subprocess.CalledProcessError as e:
            print(f"Error executing {script}: {e}")

if __name__ == "__main__":
    run_batch_processing()

--- Task 1: Batch Processing Simulation ---
Main System: Launching batch_script_1.py...
--> Script 1 is executing: Hello World
Main System: batch_script_1.py executed successfully.

Main System: Launching batch_script_2.py...
--> Script 2 is executing: Calculating...
--> Script 2 finished.
Main System: batch_script_2.py executed successfully.



In [5]:
# Task 2 - System Startup and Logging

import multiprocessing
import logging
import time
# Import the worker function from the file we created in Cell 1
from task_utils import process_task

def run_system_startup():
    print("--- Task 2: System Startup & Logging ---")
    
    # Setup logging
    logging.basicConfig(
        filename='system_log.txt', 
        level=logging.INFO,
        format='%(asctime)s - %(processName)s - %(message)s',
        force=True # Force reconfiguration in notebook
    )
    
    print("System Booting...")
    logging.info("System Boot sequence initiated...")
    
    # Create processes [cite: 206-207]
    p1 = multiprocessing.Process(target=process_task, args=("Process-A",))
    p2 = multiprocessing.Process(target=process_task, args=("Process-B",))
    
    # Start processes [cite: 208-209]
    p1.start()
    p2.start()
    
    # Wait for processes [cite: 210-211]
    p1.join()
    p2.join()
    
    logging.info("System Shutdown sequence completed.")
    print("System Shutdown. Check 'system_log.txt' for details.")

if __name__ == "__main__":
    run_system_startup()

--- Task 2: System Startup & Logging ---
System Booting...
System Shutdown. Check 'system_log.txt' for details.


In [6]:
# Task 3 - System Calls (Fork, Exec, Pipe)

import os
import sys

def run_system_calls():
    print("--- Task 3: IPC using Pipe and Fork ---")
    
    # Check if OS supports fork (Unix/Linux/Mac)
    if not hasattr(os, 'fork'):
        print("Warning: os.fork() is not available on Windows.")
        print("Skipping fork simulation. (This code requires Linux/macOS environment as per Lab Sheet).")
        return

    # Create a pipe [cite: 215]
    r, w = os.pipe()
    
    print("Parent: creating child process...")
    pid = os.fork() # [cite: 216]
    
    if pid > 0:
        # PARENT PROCESS
        os.close(r) # Close read end
        print(f"Parent (PID {os.getpid()}): Sending message to child...")
        message = b"Hello from Parent Process!"
        os.write(w, message) # [cite: 219]
        os.close(w)
        os.wait() # Wait for child to finish [cite: 221]
        print("Parent: Child process finished.")
        
    else:
        # CHILD PROCESS
        os.close(w) # Close write end
        # Read from pipe
        message_received = os.read(r, 1024) # [cite: 224]
        print(f"Child  (PID {os.getpid()}): Received -> '{message_received.decode()}'")
        os.close(r)
        sys.exit(0) # Exit child process

if __name__ == "__main__":
    run_system_calls()

--- Task 3: IPC using Pipe and Fork ---
Parent: creating child process...
Child  (PID 63338): Received -> 'Hello from Parent Process!'


SystemExit: 0

/opt/anaconda3/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


--- Task 3: IPC using Pipe and Fork ---
Parent: creating child process...
Parent (PID 63177): Sending message to child...


/var/folders/8d/36vnphm965qdnq_5vmh6n7700000gn/T/ipykernel_63177/3809148951.py:19: DeprecationWarning: This process (pid=63177) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork() # [cite: 216]


KeyboardInterrupt: 

In [7]:
# Task 4 - VM Detection & Shell Scripting

import platform
import subprocess
import os

def run_vm_detection():
    print("--- Task 4: VM Detection & Shell Interaction ---")
    
    # Part A: Create and Run Bash Script
    bash_script = """
    echo "--- Shell Script Output ---"
    echo "Kernel Version: $(uname -r)"
    echo "User: $(whoami)"
    """
    
    # Save bash script
    with open("sys_info.sh", "w") as f:
        f.write(bash_script)
        
    # Execute bash script (works on Linux/Mac/GitBash)
    try:
        subprocess.run(["bash", "sys_info.sh"], check=True)
    except FileNotFoundError:
        print("Bash not found. Skipping shell script execution.")

    # Part B: Python VM Detection Logic
    print("\n--- Python VM Detection ---")
    system_info = platform.uname()
    print(f"System: {system_info.system}")
    print(f"Node Name: {system_info.node}")
    
    # Heuristic detection for VM (checking common keywords)
    # Note: This is a basic simulation. Real detection is more complex.
    vm_keywords = ['vbox', 'vmware', 'kvm', 'virtual', 'qemu']
    is_vm = False
    
    # Check processor info or release info
    full_info = (system_info.release + system_info.version + system_info.processor).lower()
    
    for kw in vm_keywords:
        if kw in full_info:
            is_vm = True
            print(f"VM Detected! Found keyword: '{kw}'")
            break
            
    if not is_vm:
        print("No obvious VM signatures found (likely Host OS or unprivileged container).")

if __name__ == "__main__":
    run_vm_detection()

--- Task 4: VM Detection & Shell Interaction ---
--- Shell Script Output ---
Kernel Version: 25.1.0
User: sainayanmamilla

--- Python VM Detection ---
System: Darwin
Node Name: Sais-MacBook-Pro.local
No obvious VM signatures found (likely Host OS or unprivileged container).


In [8]:
# Task 5: CPU Scheduling Algorithms (Automated)

def calculate_metrics(completed_processes):
    n = len(completed_processes)
    total_wt = sum(p['wt'] for p in completed_processes)
    total_tat = sum(p['tat'] for p in completed_processes)
    print(f"Avg Waiting Time: {total_wt / n:.2f}")
    print(f"Avg Turnaround Time: {total_tat / n:.2f}")

def fcfs(processes):
    print("\n[FCFS Scheduling]")
    # Sort by arrival time (assuming arrival=0 for simplicity based on previous labs, or list order)
    current_time = 0
    print("PID\tBT\tWT\tTAT")
    for p in processes:
        wt = current_time
        tat = wt + p['bt']
        print(f"{p['id']}\t{p['bt']}\t{wt}\t{tat}")
        p['wt'] = wt
        p['tat'] = tat
        current_time += p['bt']
    calculate_metrics(processes)

def sjf(processes):
    print("\n[SJF Scheduling - Non-Preemptive]")
    # Sort by Burst Time
    sorted_p = sorted(processes, key=lambda x: x['bt'])
    current_time = 0
    print("PID\tBT\tWT\tTAT")
    for p in sorted_p:
        wt = current_time
        tat = wt + p['bt']
        print(f"{p['id']}\t{p['bt']}\t{wt}\t{tat}")
        p['wt'] = wt
        p['tat'] = tat
        current_time += p['bt']
    calculate_metrics(sorted_p)

def priority_sched(processes):
    print("\n[Priority Scheduling (Lower = Higher)]")
    # Sort by Priority
    sorted_p = sorted(processes, key=lambda x: x['pri'])
    current_time = 0
    print("PID\tBT\tPri\tWT\tTAT")
    for p in sorted_p:
        wt = current_time
        tat = wt + p['bt']
        print(f"{p['id']}\t{p['bt']}\t{p['pri']}\t{wt}\t{tat}")
        p['wt'] = wt
        p['tat'] = tat
        current_time += p['bt']
    calculate_metrics(sorted_p)

def round_robin(processes, quantum):
    print(f"\n[Round Robin Scheduling (Quantum={quantum})]")
    # Deep copy to allow modification
    queue = [{'id': p['id'], 'bt': p['bt'], 'rem': p['bt'], 'wt': 0, 'tat': 0} for p in processes]
    time = 0
    remaining = len(queue)
    
    print("PID\tBT\tWT\tTAT")
    while remaining > 0:
        for p in queue:
            if p['rem'] > 0:
                if p['rem'] > quantum:
                    time += quantum
                    p['rem'] -= quantum
                else:
                    time += p['rem']
                    p['tat'] = time
                    p['rem'] = 0
                    p['wt'] = p['tat'] - p['bt']
                    remaining -= 1
                    print(f"{p['id']}\t{p['bt']}\t{p['wt']}\t{p['tat']}")
    
    calculate_metrics(queue)

if __name__ == "__main__":
    print("--- Task 5: CPU Scheduling Automation ---")
    
    # Hardcoded Test Data [PID, BurstTime, Priority]
    data = [
        {'id': 1, 'bt': 10, 'pri': 2},
        {'id': 2, 'bt': 5,  'pri': 1}, # Highest priority
        {'id': 3, 'bt': 8,  'pri': 3}
    ]
    
    # Run algorithms
    fcfs([d.copy() for d in data])
    sjf([d.copy() for d in data])
    priority_sched([d.copy() for d in data])
    round_robin([d.copy() for d in data], quantum=4)

--- Task 5: CPU Scheduling Automation ---

[FCFS Scheduling]
PID	BT	WT	TAT
1	10	0	10
2	5	10	15
3	8	15	23
Avg Waiting Time: 8.33
Avg Turnaround Time: 16.00

[SJF Scheduling - Non-Preemptive]
PID	BT	WT	TAT
2	5	0	5
3	8	5	13
1	10	13	23
Avg Waiting Time: 6.00
Avg Turnaround Time: 13.67

[Priority Scheduling (Lower = Higher)]
PID	BT	Pri	WT	TAT
2	5	1	0	5
1	10	2	5	15
3	8	3	15	23
Avg Waiting Time: 6.67
Avg Turnaround Time: 14.33

[Round Robin Scheduling (Quantum=4)]
PID	BT	WT	TAT
2	5	12	17
3	8	13	21
1	10	13	23
Avg Waiting Time: 12.67
Avg Turnaround Time: 20.33
